In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import glob
from joblib import Parallel, delayed
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns',300)
pd.set_option('max_rows',500)

# Calculation of Basic Characteristic Variables

### Calculate WAP
$$ \text{wap} = \frac{\text{bid_price} * \text{ask_size} + \text{ask_price} * \text{bid_size}}{\text{bid_size} + \text{ask_size}} $$
### Calculate log return
$$ \text{log return}_{t_1,t_2} = LR_{t_1,t_2} = \log{\frac{S_{t_2}}{S_{t_1}}}  = \log{S_{t_2}} - \log{S_{t_1}}$$ 
### Calculate realized volatility
$$ \text{realized volatility} = \sigma = \sqrt{\sum_t{LR_{t-1,t}^2}} $$

In [ ]:
def calculate_wap(df,method):
    if method == 1:
        wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    if method == 2:
        wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def log_return(series):
    return np.log(series).diff()

def realized_volatility(series):
    series = log_return(series)
    return np.sqrt(np.sum(series**2))

### Count Unique
To count how many unique elements in the series

In [ ]:
def count_unique(series):
    return len(np.unique(series))

# Feature Consideration

### 1. The realized volatility $\sigma$ of the past 10 mins
Use the wap to calculate log return, then we can calculate $\sigma$  
**Be aware:** each time record(**seconds_in_bucket**) will have a **wap**, **log return** corresponding to it, while each **volatility $\sigma$** only corresponds to one **time_id**
### 2. Statistics(sum, mean, std) of some distribution information
(1) **wap1**
$$ \text{wap1} = \frac{\text{bid_price}_1 * \text{ask_size}_1 + \text{ask_price}_1 * \text{bid_size}_1}{\text{bid_size}_1 + \text{ask_size}_1} $$
(2) **wap2**
$$ \text{wap2} = \frac{\text{bid_price}_2 * \text{ask_size}_2 + \text{ask_price}_2 * \text{bid_size}_2}{\text{bid_size}_2 + \text{ask_size}_2} $$
<font color=red>**wap1** and **wap2** are the highest and second highest stock Weighted Average Price </font>  
<font color=blue>You can find the relevant information and evidence in **Figure 1** </font>  
(3) **log_return1**
$$ \text{log_return}_{t_1,t_2}^{(1)} = \log{\frac{\text{wap1}_{t_2}}{\text{wap1}_{t_1}}}  = \log{\text{wap1}_{t_2}} - \log{\text{wap1}_{t_1}}$$ 
(4) **log_return2**
$$ \text{log_return}_{t_1,t_2}^{(2)} = \log{\frac{\text{wap2}_{t_2}}{\text{wap2}_{t_1}}}  = \log{\text{wap2}_{t_2}} - \log{\text{wap2}_{t_1}}$$ 
<font color=red>If the volatility $\sigma$ is high, we can believe **log_return1** and **log_return2** will have high variance </font>  
<font color=blue>You can find the relevant information and evidence in **Figure 2** </font>    
(5) **wap_balance**
$$ \text{wap_balance} = |\text{wap1} - \text{wap2}| $$
(6) **price_spread**
$$ \text{price_spread} = \frac{\text{ask_price}_1 - \text{bid_price}_1}{\frac{\text{ask_price}_1 + \text{bid_price}_1}{2}} 
= \frac{2(\text{ask_price}_1 - \text{bid_price}_1)}{\text{ask_price}_1 + \text{bid_price}_1}$$
(7) **bid_spread**
$$ \text{bid_spread} = \text{bid_price}_1 - \text{bid_price}_2 $$
(8) **ask_spread**
$$ \text{ask_spread} = \text{ask_price}_1 - \text{ask_price}_2 $$
<font color=red>A stock has high **volatility $\sigma$** may have a big gap in **wap_balance, price_spread, bid_spread, ask_spread**</font>  
<font color=blue>You can find the relevant information and evidence in **Figure 3** </font>  
(9) **total_volume**
$$ \text{total_volume} = \text{ask_size}_1 + \text{ask_size}_2 + \text{bid_size}_1 + \text{bid_size}_2 $$
(10) **volume_imbalance**
$$ \text{volume_imbalance} = |(\text{ask_size}_1 + \text{ask_size}_2) - (\text{bid_size}_1 + \text{bid_size}_2)| $$
<font color=blue>You can find the relevant information and evidence in **Figure 4** </font>  
### 3. Consider similar features in **Trade**
(1) **realized_volatility**  
(2) Statistics of **price, log_return, size, order_count**<font color=blue> (shown in **Figure 5**)</font>  
(3) **count_unique** of **seconds_in_bucket**, **(That is, how many actual transactions have occurred in the trade)**  
<font color=red>It can be considered that **a frequent transaction means higher volatility**</font>
### 4. Handling **stock_id** and **time_id**
(1) Use the **statistics** of historical **volatility $\sigma$** corresponding to **stock_id** and **time_id** to represent their features  
(2) Use **Target Encoding(TE)** to represent the feature of **stock_id, time_id**  
(3) Use a **Embedding** layer in NN to generate feature vectors
(4) Use **stock_id, time_id** as **category variables**
### 5. Feature Scaling
This can eliminate the influence of some numerical fluctuations, but it may not be effective
### 6. Time Series Analysis
We may use **historical time series** as features to directly predict **future series** or **future volatility**

In [ ]:
stock_id = 0
book = pd.read_parquet('../input/optiver-realized-volatility-prediction/book_train.parquet/stock_id='+str(stock_id))
book['wap1'] = calculate_wap(book,1)
book_sub = book.groupby(['time_id'])['wap1'].agg(realized_volatility).to_frame().reset_index()
book_sub.columns = ['time_id','realized volatility']
book_sub.head()

### Fetch the first two times_id(time_id=5, 11) to compare the relationship between volatility $\sigma$ and Features

In [ ]:
book['wap2'] = calculate_wap(book,2)
book['log_return1'] = book.groupby(['time_id'])['wap1'].apply(log_return)
book['log_return2'] = book.groupby(['time_id'])['wap2'].apply(log_return)
book['wap_balance'] = abs(book['wap1'] - book['wap2'])
book['price_spread'] = (book['ask_price1'] - book['bid_price1']) / ((book['ask_price1'] + book['bid_price1']) / 2)
book['bid_spread'] = book['bid_price1'] - book['bid_price2']
book['ask_spread'] = book['ask_price1'] - book['ask_price2']
book['total_volume'] = (book['ask_size1'] + book['ask_size2']) + (book['bid_size1'] + book['bid_size2'])
book['volume_imbalance'] = abs((book['ask_size1'] + book['ask_size2']) - (book['bid_size1'] + book['bid_size2']))
book_5 = book.loc[book['time_id']==5]
book_11 = book.loc[book['time_id']==11]
book_5.head()

### Figure 1: **wap1** and **wap2**

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(18,5))
axes[0].plot(book_5['seconds_in_bucket'],book_5['wap1'],label='time_id:5')
axes[0].plot(book_11['seconds_in_bucket'],book_11['wap1'],label='time_id:11')
axes[0].set_xlabel('seconds_in_bucket')
axes[0].set_ylabel('WAP')
axes[0].set_title('WAP1')
axes[0].legend()
axes[1].plot(book_5['seconds_in_bucket'],book_5['wap2'],label='time_id:5')
axes[1].plot(book_11['seconds_in_bucket'],book_11['wap2'],label='time_id:11')
axes[1].set_xlabel('seconds_in_bucket')
axes[1].set_ylabel('WAP')
axes[1].set_title('WAP2')
axes[1].legend()

<font color=red>From **Figure 1**, we can clearly see the correlation between **volatility $\sigma$** and **wap1, wap2**. The transformation of WAP of stocks with low volatility is very gentle</font>

### Figure 2: **log_return1** and **log_return2**

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(18,5))
axes[0].plot(book_5['seconds_in_bucket'],book_5['log_return1'],label='time_id:5')
axes[0].plot(book_11['seconds_in_bucket'],book_11['log_return1'],label='time_id:11')
axes[0].set_xlabel('seconds_in_bucket')
axes[0].set_ylabel('log_return')
axes[0].set_title('log_return1')
axes[0].legend()
axes[1].plot(book_5['seconds_in_bucket'],book_5['log_return2'],label='time_id:5')
axes[1].plot(book_11['seconds_in_bucket'],book_11['log_return2'],label='time_id:11')
axes[1].set_xlabel('seconds_in_bucket')
axes[1].set_ylabel('log_return')
axes[1].set_title('log_return1')
axes[1].legend()

<font color=red>From **Figure 2**, we can directly observe the performance of **volatility $\sigma$** in **log_return**. Besides the value of $\sigma$ in the past 10 mins, the statistics of log_return may be helpful</font>

### Figure 3: **wap_balance, price_spread, bid_spread, ask_spread**

In [ ]:
fig,axes = plt.subplots(2,2,figsize=(18,12))
axes = axes.flatten()
feature_dict = ['wap_balance','price_spread','bid_spread','ask_spread']
for i in range(4):
    axes[i].plot(book_5['seconds_in_bucket'],book_5[feature_dict[i]],label='time_id:5')
    axes[i].plot(book_11['seconds_in_bucket'],book_11[feature_dict[i]],label='time_id:11')
    axes[i].set_xlabel('seconds_in_bucket')
    axes[i].set_ylabel(feature_dict[i])
    axes[i].set_title(feature_dict[i])
    axes[i].legend()

<font color=red>From **Figure 3**, we can find little correlation between **volatility $\sigma$** and those four features. They show some trends, but not so strong. We can still add them to the features of the model. It seems that the stock with high **volatility** has high variance, extreme values, high gap in these four features</font>

### Figure 4: **total_volume** and **valume_inbalance**

In [ ]:
fig,axes = plt.subplots(1,2,figsize=(18,5))
axes[0].plot(book_5['seconds_in_bucket'],book_5['total_volume'],label='time_id:5')
axes[0].plot(book_11['seconds_in_bucket'],book_11['total_volume'],label='time_id:11')
axes[0].set_xlabel('seconds_in_bucket')
axes[0].set_ylabel('total_volume')
axes[0].set_title('total_volume')
axes[0].legend()
axes[1].plot(book_5['seconds_in_bucket'],book_5['volume_imbalance'],label='time_id:5')
axes[1].plot(book_11['seconds_in_bucket'],book_11['volume_imbalance'],label='time_id:11')
axes[1].set_xlabel('seconds_in_bucket')
axes[1].set_ylabel('volume_imbalance')
axes[1].set_title('volume_imbalance')
axes[1].legend()

<font color=red>From **Figure 4**, we hardly see the correlations between **volatility $\sigma$** and **total_volume, volume_imbalance**</font>

In [ ]:
stock_id = 0
trade = pd.read_parquet('../input/optiver-realized-volatility-prediction/trade_train.parquet/stock_id='+str(stock_id))
trade.head()

In [ ]:
trade_sub = trade.groupby(['time_id'])['price'].agg(realized_volatility).to_frame().reset_index()
trade_sub.columns = ['time_id','realized volatility']
trade_sub.head()

In [ ]:
trade['log_return'] = trade.groupby(['time_id'])['price'].apply(log_return)
trade_5 = trade.loc[trade['time_id']==5]
trade_62 = trade.loc[trade['time_id']==62]
trade_5.head()

### Figure 5: **price, log_return, size, order_count** in **Trade**

In [ ]:
fig,axes = plt.subplots(2,2,figsize=(18,12))
axes = axes.flatten()
feature_dict = ['price','log_return','size','order_count']
for i in range(4):
    axes[i].plot(trade_5['seconds_in_bucket'],trade_5[feature_dict[i]],label='time_id:5')
    axes[i].plot(trade_62['seconds_in_bucket'],trade_62[feature_dict[i]],label='time_id:62')
    axes[i].set_xlabel('seconds_in_bucket')
    axes[i].set_ylabel(feature_dict[i])
    axes[i].set_title(feature_dict[i])
    axes[i].legend()

<font color=red>From **Figure 5**, the difference of **volatility $\sigma$** represented by these characteristics is not obvious. We can still use them to build the model, but there may be more key information hidden in **Trade**</font>

## Capture the features of the trend
We can use a window to select the **information closer to the future**, and obtain the **change trend of features** over time through multiple windows

In [ ]:
def get_stats_window(seconds_in_bucket, add_suffix = False):
    df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket]
    # add seconds_in_bucket suffix
    if add_suffix:
        df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
    return df_feature

# Feature Generation & Function Encapsulation

### Accelerate data processing with <font color=red>**numba**</font>

In [ ]:
def calculate_wap(df,method):
    if method == 1:
        wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    if method == 2:
        wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def log_return(series):
    return np.log(series).diff()

### Functions used in **numba**

In [ ]:
def realized_volatility_numba(values,index=None):
    def log_return_in_realized_volatility(series):
        return np.diff(np.log(series))
    LR = log_return_in_realized_volatility(values)
    return np.sqrt(np.sum(np.square(LR)))

def mean_numba(values,index=None):
    return np.mean(values)

def sum_numba(values,index=None):
    return np.sum(values)

def std_numba(values,index=None):
    return np.std(values)

def min_numba(values,index=None):
    return np.min(values)

def max_numba(values,index=None):
    return np.max(values)

def max_min_numba(values,index=None):
    return np.max(values) - np.min(values)

def count_unique_numba(values,index=None):
    return len(np.unique(values))

## Performance Comparison

### <font color=red>**FeatureEngine** in traditional way</font>

In [ ]:
def book_feature(file_path):
    df = pd.read_parquet(file_path)
    df['wap1'] = calculate_wap(df,1)
    df['wap2'] = calculate_wap(df,2)
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    create_feature_dict = {
        'wap1': [np.max, np.min, realized_volatility, np.mean, np.std],
        'wap2': [np.max, np.min, realized_volatility, np.mean, np.std],
        'log_return1': [np.max, np.min, np.mean, np.std],
        'log_return2': [np.max, np.min, np.mean, np.std],
        'wap_balance': [np.max, np.min, np.mean, np.std],
        'price_spread':[np.max, np.min, np.mean, np.std],
        'bid_spread':[np.max, np.min, np.mean, np.std],
        'ask_spread':[np.max, np.min, np.mean, np.std],
        'total_volume':[np.max, np.min, np.mean, np.std],
        'volume_imbalance':[np.max, np.min, np.mean, np.std]
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    windows = []
    seconds = [450, 300, 150]
    time_id_list = ['time_id__'+str(second) for second in seconds]
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    for second in seconds:
        windows.append(get_stats_window(seconds_in_bucket = second, add_suffix = True))
    for i,second in enumerate(seconds):
        df_feature = df_feature.merge(windows[i], how = 'left', left_on = 'time_id_', right_on = time_id_list[i])
    df_feature.drop(time_id_list, axis = 1, inplace = True)
    
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['time_id_'], axis = 1, inplace = True)
    return df_feature

def trade_feature(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby(['time_id'])['price'].apply(log_return)
    create_feature_dict = {
        'price': [np.max, np.min, realized_volatility, np.mean, np.std],
        'log_return': [np.max, np.min, np.mean, np.std],
        'seconds_in_bucket': [count_unique],
        'size': [np.max, np.min, np.mean, np.std],
        'order_count': [np.max, np.min, np.mean, np.std]
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        df_feature = df[df['seconds_in_bucket'] >= seconds_in_bucket].groupby(['time_id']).agg(create_feature_dict).reset_index()
        df_feature.columns = ['_'.join(col) for col in df_feature.columns]
        if add_suffix:
            df_feature = df_feature.add_suffix('_' + str(seconds_in_bucket))
        return df_feature
    
    windows = []
    seconds = [450, 300, 150]
    time_id_list = ['time_id__'+str(second) for second in seconds]
    df_feature = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    for second in seconds:
        windows.append(get_stats_window(seconds_in_bucket = second, add_suffix = True))
    for i,second in enumerate(seconds):
        df_feature = df_feature.merge(windows[i], how = 'left', left_on = 'time_id_', right_on = time_id_list[i])
    df_feature.drop(time_id_list, axis = 1, inplace = True)
    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x: f'{stock_id}-{x}')
    df_feature.drop(['trade_time_id_'], axis = 1, inplace = True)
    return df_feature

### <font color=red>**FeatureEngine** with **numba** engine</font>

In [ ]:
def book_feature_numba(file_path):
    df = pd.read_parquet(file_path)
    df['wap1'] = calculate_wap(df,1)
    df['wap2'] = calculate_wap(df,2)
    df['log_return1'] = df.groupby(['time_id'])['wap1'].apply(log_return)
    df['log_return2'] = df.groupby(['time_id'])['wap2'].apply(log_return)
    df['wap_balance'] = abs(df['wap1'] - df['wap2'])
    df['price_spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1']) / 2)
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    df.fillna(0,inplace=True)
    method_dict = {
        'max': max_numba,
        'min': min_numba,
        'mean': mean_numba,
        'std': std_numba
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        df_window = df[df['seconds_in_bucket'] >= seconds_in_bucket]
        if add_suffix:
            df_window = df_window.add_suffix('_' + str(seconds_in_bucket))
        return df_window
    
    def get_feature(df_temp,time_id,suffix=None):
        rv_list = ['wap1','wap2']
        statistics_list = ['wap1','wap2','log_return1','log_return2',
                           'wap_balance','price_spread','bid_spread','ask_spread',
                           'total_volume','volume_imbalance']
        if suffix != None:
            rv_list = [x + '_' + str(suffix) for x in rv_list]
            statistics_list = [x + '_' + str(suffix) for x in statistics_list]
        df_feature = df_temp.groupby([time_id])[rv_list].agg(realized_volatility_numba,engine='numba').reset_index()
        df_feature.columns = [col if col == time_id else col + '_rv' for col in df_feature.columns]
        for method in method_dict:
            merge_in = df_temp.groupby([time_id])[statistics_list].agg(method_dict[method],engine='numba').reset_index()
            merge_in.columns = [col if col == time_id else (col + '_'+ method) for col in merge_in.columns]
            df_feature = df_feature.merge(merge_in,how='left',left_on=time_id,right_on=time_id)
        return df_feature
    
    windows = []
    seconds = [450, 300, 150]
    time_id_list = ['time_id_'+ str(second) for second in seconds] 
    df_all = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_all = get_feature(df_all,'time_id')
    for second in seconds:
        windows.append(get_stats_window(seconds_in_bucket = second, add_suffix = True))
    for i,second in enumerate(seconds):
        df_all = df_all.merge(get_feature(windows[i], time_id_list[i], second), 
                              how = 'left', left_on = 'time_id', right_on = time_id_list[i])
    df_all.drop(time_id_list, axis = 1, inplace = True)
    
    stock_id = file_path.split('=')[1]
    df_all['row_id'] = df_all['time_id'].apply(lambda x: f'{stock_id}-{x}')
    df_all.drop(['time_id'], axis = 1, inplace = True)
    return df_all

def trade_feature_numba(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby(['time_id'])['price'].apply(log_return)
    df.fillna(0,inplace=True)
    method_dict = {
        'max': max_numba,
        'min': min_numba,
        'mean': mean_numba,
        'std': std_numba
    }
    
    def get_stats_window(seconds_in_bucket, add_suffix = False):
        df_window = df[df['seconds_in_bucket'] >= seconds_in_bucket]
        if add_suffix:
            df_window = df_window.add_suffix('_' + str(seconds_in_bucket))
        return df_window
    
    def get_feature(df_temp,time_id,suffix=None):
        rv_list = ['price']
        statistics_list = ['price','log_return','size','order_count']
        count_unique_list = ['seconds_in_bucket']
        if suffix != None:
            rv_list = [x + '_' + str(suffix) for x in rv_list]
            statistics_list = [x + '_' + str(suffix) for x in statistics_list]
            count_unique_list = [x + '_' + str(suffix) for x in count_unique_list]
        df_feature = df_temp.groupby([time_id])[rv_list].agg(realized_volatility_numba,engine='numba').reset_index()
        df_feature.columns = [col if col == time_id else col + '_rv' for col in df_feature.columns]
        for method in method_dict:
            merge_in = df_temp.groupby([time_id])[statistics_list].agg(method_dict[method],engine='numba').reset_index()
            merge_in.columns = [col if col == time_id else (col + '_'+ method) for col in merge_in.columns]
            df_feature = df_feature.merge(merge_in,how='left',left_on=time_id,right_on=time_id)
        merge_in = df_temp.groupby([time_id])[count_unique_list].agg(count_unique_numba,engine='numba').reset_index()
        merge_in.columns = [col if col == time_id else (col + '_count_unique') for col in merge_in.columns]
        df_feature = df_feature.merge(merge_in,how='left',left_on=time_id,right_on=time_id)
        return df_feature
    
    windows = []
    seconds = [450, 300, 150]
    time_id_list = ['time_id_'+str(second) for second in seconds] 
    df_all = get_stats_window(seconds_in_bucket = 0, add_suffix = False)
    df_all = get_feature(df_all,'time_id')
    for second in seconds:
        windows.append(get_stats_window(seconds_in_bucket = second, add_suffix = True))
    for i,second in enumerate(seconds):
        df_all = df_all.merge(get_feature(windows[i], time_id_list[i], second), 
                              how = 'left', left_on = 'time_id', right_on = time_id_list[i])
    df_all.drop(time_id_list, axis = 1, inplace = True)
    
    df_all = df_all.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_all['row_id'] = df_all['trade_time_id'].apply(lambda x: f'{stock_id}-{x}')
    df_all.drop(['trade_time_id'], axis = 1, inplace = True)
    return df_all

In [ ]:
def FeatureEngineering(list_stock_ids,is_train = True,use_numba=False):
    def for_joblib(stock_id):
        data_dir = '../input/optiver-realized-volatility-prediction/'
        # Train
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        # book
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
        if use_numba:
            df_temp = pd.merge(book_feature_numba(file_path_book), 
                               trade_feature_numba(file_path_trade), how='left', on='row_id')
        else:
            df_temp = pd.merge(book_feature(file_path_book), 
                               trade_feature(file_path_trade), how='left', on='row_id')
        return df_temp
    
    df = Parallel(n_jobs = -1, verbose = 1)(delayed(for_joblib)(stock_id) for stock_id in list_stock_ids)
    df = pd.concat(df, ignore_index = True)
    return df

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
train_stock_ids = train['stock_id'].unique()

In [ ]:
%%time
train_feature = FeatureEngineering(train_stock_ids, is_train = True, use_numba=False)

In [ ]:
%%time
train_feature_numba = FeatureEngineering(train_stock_ids, is_train = True, use_numba=True)

### <font color=red>From above, we can find **numba** engine can save more than 60% time in the process of feature generation</font>

### Save features for later use

In [ ]:
train_feature_numba.to_csv('train_feature.csv',index=False)

In [ ]:
print(train_feature_numba.shape)

## Handling **stock_id** and **time_id**

### (1) Use **statistics** of **volatility $\sigma$** to represent the feature of **stock_id, time_id**

In [ ]:
def statistics_rv_time_stock(df):
    vr_cols = []
    for col in df.columns:
        if 'rv' in col:
            vr_cols.append(col)

    df_stock_id = df.groupby(['stock_id'])[vr_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_stock_id.columns = ['_'.join(col) for col in df_stock_id.columns]
    df_stock_id = df_stock_id.add_suffix('_' + 'stock')

    df_time_id = df.groupby(['time_id'])[vr_cols].agg(['mean', 'std', 'max', 'min', ]).reset_index()
    df_time_id.columns = ['_'.join(col) for col in df_time_id.columns]
    df_time_id = df_time_id.add_suffix('_' + 'time')
    
    df = df.merge(df_stock_id, how = 'left', left_on = 'stock_id', right_on = ['stock_id__stock'])
    df = df.merge(df_time_id, how = 'left', left_on = 'time_id', right_on = ['time_id__time'])
    df.drop(['stock_id__stock', 'time_id__time'], axis = 1, inplace = True)
    return df

### (2) Use **Target Encoding(TE)**
We can use the **mean value** of the **target volatility $\sigma$** to represent the feature of **stock_id** (Because the same stock series has strong **autocorrelation**)  

<font color=red>**Be ware:** Since the distribution of test set and training set may be different, we use **TE** with noise</font>

In [ ]:
from sklearn.model_selection import KFold
def stock_id_TE(df_train,df_test):
    if 'stock_id' not in df_train.columns:
        df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
    if 'stock_id' not in df_test.columns:
        df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])
    stock_id_target_mean = df_train.groupby('stock_id')['target'].mean()
    df_test['stock_id_target_encoding'] = df_test['stock_id'].map(stock_id_target_mean)
    # TE with noise in training set
    tmp = np.repeat(np.nan, df_train.shape[0])
    kf = KFold(n_splits = 10, shuffle=True,random_state = 19911109)
    for idx_1, idx_2 in kf.split(df_train):
        target_mean = df_train.iloc[idx_1].groupby('stock_id')['target'].mean()
        tmp[idx_2] = df_train['stock_id'].iloc[idx_2].map(target_mean)
    df_train['stock_id_target_enc'] = tmp
    return df_train, df_test

### (3) Use **Embedding Layer**

In [ ]:
hidden_units = (32,16)
stock_embedding_size = 16
def Embedding_NN(train,hidden_units,stock_embedding_size,num_shape,cat_shape):
    cat_data = train['stock_id']
    stock_id_input = keras.Input(shape=(1,), name='stock_id')
    num_input = keras.Input(shape=(3,), name='num_data')
    
    stock_embedded = keras.layers.Embedding(max(cat_data)+1, stock_embedding_size, 
                                            input_length=1, name='stock_embedding')(stock_id_input)
    stock_flattened = keras.layers.Flatten()(stock_embedded)
    out = keras.layers.Concatenate()([stock_flattened, num_input])
    
    for n_hidden in hidden_units:
        out = keras.layers.Dense(n_hidden, activation='selu')(out)
    out = keras.layers.Dense(1, activation='linear', name='prediction')(out)
    
    model = keras.Model(
        inputs = [stock_id_input, num_input],
        outputs = out,
    )
    return model

### (4) Use **stock_id, time_id** as **category variables**

In [ ]:
def category_stock(df_train,df_test):
    df_train['stock_id'].astype('cat')
    df_test['stock_id'].astype('cat')
    return df_train,df_test

## Feature Scaling

In [ ]:
def feature_scale(df_train,df_test,features,method=None):
    for f in features:
        if method == 'min_max':
            minimun = df_train[f].min()
            maximum = df_train[f].max()
            df_train[f] = (df_train[f] - minimum) / (maximum - minimum)
            df_test[f] = (df_test[f] - minimum) / (maximum - minimum)
        if method == 'z_score':
            minimum = df_train[f].min()
            std = df_train[f].std()
            df_train[f] = (df_train[f] - minimum) / std
            df_test[f] = (df_test[f] - minimum) / std
    return df_train, df_test

# Correlation Between Target and Feature

### Use **TE** to handling **stock_id, time_id**

In [ ]:
train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
test_stock_ids = test['stock_id'].unique()
test_feature = FeatureEngineering(test_stock_ids, is_train = False, use_numba=True)
train_feature = pd.read_csv('../input/optiver-train-data/train_feature.csv')
train_feature.fillna(method='ffill',inplace=True)
train_feature.fillna(0,inplace=True)

In [ ]:
train_feature['stock_id'] = train_feature['row_id'].apply(lambda x: x.split('-')[0]).astype('int64')
train_feature['time_id'] = train_feature['row_id'].apply(lambda x: x.split('-')[1]).astype('int64')
train_feature = train_feature.merge(train, how='left', on=['stock_id','time_id'])
test_feature = test_feature.merge(test, how='right', on=['row_id'])
test_feature.fillna(method='ffill',inplace=True)
test_feature.fillna(0,inplace=True)
train_feature, test_feature = stock_id_TE(train_feature,test_feature)

### Fetch features' name

In [ ]:
features = []
for col in train_feature.columns:
    if col not in ['stock_id','time_id','row_id','target']:
        features.append(col)

### Calculate correlations

In [ ]:
corr_matrix = train_feature.corr()
corr_target = corr_matrix['target']
corr_target = corr_target.sort_values(ascending = False)
corr_target = corr_target.to_frame()

In [ ]:
corr_target.head(500)

In [ ]:
g1_features = []
for col in train_feature.columns:
    if 'rv' in col:
        g1_features.append(col)
corr_target.loc[g1_features,]

In [ ]:
train_feature[g1_features].head(5)

In [ ]:
g2_features = []
for col in train_feature.columns:
    if 'std' in col and 'log_return' in col:
        g2_features.append(col)
corr_target.loc[g2_features,]

In [ ]:
train_feature[g2_features].head(5)

In [ ]:
g3_features = []
for col in train_feature.columns:
    if 'max' in col and 'log_return' in col:
        g3_features.append(col)
corr_target.loc[g3_features,]

In [ ]:
train_feature[g3_features].head(5)

In [ ]:
g4_features = []
for col in train_feature.columns:
    if 'min' in col and 'log_return' in col:
        g4_features.append(col)
corr_target.loc[g4_features,]

In [ ]:
train_feature[g4_features].head(5)

In [ ]:
g5_features = []
for col in train_feature.columns:
    if 'price_spread' in col and 'min' not in col:
        g5_features.append(col)
corr_target.loc[g5_features,]

In [ ]:
train_feature[g5_features].head(5)

In [ ]:
g6_features = []
for col in train_feature.columns:
    if 'wap_balance' in col and 'min' not in col:
        g6_features.append(col)
corr_target.loc[g6_features,]

In [ ]:
train_feature[g6_features].head(5)

In [ ]:
g7_features = []
for col in train_feature.columns:
    if 'wap' in col and 'std' in col and 'wap_balance' not in col:
        g7_features.append(col)
corr_target.loc[g7_features,]

In [ ]:
train_feature[g7_features].head(5)

In [ ]:
g8_features = []
for col in train_feature.columns:
    if 'bid_spread' in col and ('max' in col or 'std' in col):
        g8_features.append(col)
corr_target.loc[g8_features,]

In [ ]:
train_feature[g8_features].head(5)

### Function to plot featurn with **stock_id**

In [ ]:
from tqdm.auto import tqdm
def plot_feature_with_stock(df,feature):
    fig, ax = plt.subplots(16, 7, figsize=(20, 50))
    ax = ax.flatten()
    for i, stock_id in tqdm(enumerate(df['stock_id'].unique())):
        ax[i].scatter(df.query('stock_id == @stock_id')[feature],
                      df.query('stock_id == @stock_id')['target'],
                      s=3,c='r',alpha=0.3)
        ax[i].set_title("stock_id: " + str(stock_id))
    plt.tight_layout()

#### <font color=red>Group according to different **stock_id** and **correlation coefficient** from high to low</font>  
#### <font color=red>Plot the relationship between **feature** and **target**</font>

In [ ]:
print(corr_target.index[1])
plot_feature_with_stock(train_feature,corr_target.index[1])

In [ ]:
print(corr_target.index[2])
plot_feature_with_stock(train_feature,corr_target.index[2])

In [ ]:
print(corr_target.index[240])
plot_feature_with_stock(train_feature,corr_target.index[240])

### Function to plot **stock** with featurn

In [ ]:
import matplotlib.colors as mcolors
colors = list(mcolors.TABLEAU_COLORS.keys())
def plot_stock_with_feature(df,stock_id):
    fig, ax = plt.subplots(30, 8, figsize=(20, 80))
    ax = ax.flatten()
    for i in range(240):
        ax[i].scatter(df.query('stock_id == @stock_id')[corr_target.index[i+1]],
                      df.query('stock_id == @stock_id')['target'],
                      s=3,c=mcolors.TABLEAU_COLORS[colors[i%10]],alpha=0.3)
        ax[i].set_title(corr_target.index[i+1])
    plt.tight_layout()

In [ ]:
stock_id = 0
print("stock_id: ",stock_id)
plot_stock_with_feature(train_feature,stock_id)

In [ ]:
stock_id = 1
print("stock_id: ",stock_id)
plot_stock_with_feature(train_feature,stock_id)

### <font color=red>We can find some useful imformation from above</font>
<font color=red>(1) The **mean value** of variables do not seem to have a linear correlation with the **target** </font>  
<font color=red>(2) The **min value** of variables seem to have a negative linear correlation with the **target** </font>  
<font color=red>(3) The **most important varialbes** might be **realized volatility** in the past and **statistics of log_return**. They all have a strong positive correlation with **target**</font>  
<font color=red>(4) ......</font>

# Mutual Information

# FeatureGen

#### Now, we can package all functions related to feature engineering

In [ ]:
def FeatureGen(train_load=False,use_TE=False,use_rv_statistics=False,use_category=False,use_scale=None):
    train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
    test = pd.read_csv('../input/optiver-realized-volatility-prediction/test.csv')
    train_stock_ids = train['stock_id'].unique()
    test_stock_ids = test['stock_id'].unique()
    # If train_data already saved, we can load it directly to save time in FeatureGen
    if train_load:
        train_feature = pd.read_csv('../input/optiver-train-data/train_feature.csv')
    else:
        train_feature = FeatureEngineering(train_stock_ids, is_train = True, use_numba=True)
    test_feature = FeatureEngineering(test_stock_ids, is_train = False, use_numba=True)
    
    train_feature['stock_id'] = train_feature['row_id'].apply(lambda x: x.split('-')[0]).astype('int64')
    train_feature['time_id'] = train_feature['row_id'].apply(lambda x: x.split('-')[1]).astype('int64')
    train_feature = train_feature.merge(train, how='left', on=['stock_id','time_id'])
    test_feature = test_feature.merge(test, how='right', on=['row_id'])
    test_feature.fillna(method='ffill',inplace=True)
    test_feature.fillna(0,inplace=True)
    # Whether to use Target Encoding
    if use_TE:
        train_feature, test_feature = stock_id_TE(train_feature,test_feature)
    # Whether to use realized volatility statistics
    if use_rv_statistics:
        train_feature = statistics_rv_time_stock(train_feature)
        test_feature = statistics_rv_time_stock(test_feature)
    # Whether to make stock_id be category variable
    if use_category:
        train_feature, test_feature = category_stock(train_feature,test_feature)
    # Whether to use feature scaling
    featurns_scaling = []
    if use_scale != None:
        train_feature, test_feature = feature_scale(train_feature,test_feature,features_scaling,method=use_scale)
    
    return train_feature, test_featurn